Loading and converting keras to tensorflow pb

In [1]:
from keras import backend as K

Using TensorFlow backend.
C:\Users\Feng\Miniconda3\envs\aind-vui\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
K.set_learning_phase(0)

In [9]:
from keras.models import load_model
model = load_model('./model/keras_end_model.h5')


In [4]:
print(model.outputs)
print(model.inputs[0].shape)


[<tf.Tensor 'softmax/truediv:0' shape=(?, ?, 29) dtype=float32>]
(?, ?, 13)


In [5]:
import tensorflow as tf

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph


frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

INFO:tensorflow:Froze 22 variables.
Converted 22 variables to const ops.


In [6]:
# Save to ./model/tf_model.pb
tf.train.write_graph(frozen_graph, "model", "model_end.pb", as_text=False)

'model\\model_end.pb'

In [10]:
import platform
is_win = 'windows' in platform.platform().lower()

# OpenVINO 2019
if is_win:
    mo_tf_path = '"C:\Program Files (x86)\IntelSWTools\openvino\deployment_tools\model_optimizer\mo_tf.py"'
else:
    # mo_tf.py path in Linux
    mo_tf_path = '/opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py'

In [23]:
!mkdir rnn_1_while_mul_1

In [28]:
!pip install cogapp

  Using cached cogapp-3.0.0-py2.py3-none-any.whl (27 kB)


In [ ]:
!python "C:/Program Files (x86)/IntelSWTools/openvino/deployment_tools/tools/extension_generator/extgen.py" new --mo-tf-ext --mo-op --ie-cpu-ext --ie-gpu-ext --output_dir=rnn_1_while_mul_1

In [21]:
input_shape = model.inputs[0].shape
input_shape_str = str((1,584,13)).replace(' ','').replace('?', 'None')
pb_file = './model/model_end.pb'
output_dir = './model'

!python {mo_tf_path} --input_model {pb_file} --output_dir {output_dir} --input_shape {input_shape_str} --data_type FP32

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	C:\Users\Feng\optimizekerasmodel\./model/model_end.pb
	- Path for generated IR: 	C:\Users\Feng\optimizekerasmodel\./model
	- IR output name: 	model_end
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	(1,584,13)
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	False
TensorFlow specific parameters:
	- Input model in text protobuf format: 	False
	- Path to model dump for TensorBoard: 	None
	- List of shared libraries with TensorFlow custom layers implementation: 	None
	- Update the configuration file with input/output node names: 	None
	- Use c

C:\Users\Feng\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
[ ERROR ]  Something bad has happened with graph! Data node "rnn_1/while/mul_1" has 0 producers


In [12]:
!pip install tensorflow==1.2.0

  Using cached backports.weakref-1.0rc1-py3-none-any.whl (4.3 kB)
  Using cached Markdown-2.2.0.tar.gz (236 kB)
  Created wheel for markdown: filename=Markdown-2.2.0-py3-none-any.whl size=136956 sha256=3e16abb42500c9d581fab7f929c1ab0f37e09a578f42e7174b96c882c81a8957
  Stored in directory: c:\users\feng\appdata\local\pip\cache\wheels\ca\3a\29\594713d45f4f387ff7ba5e713398406dc0fd4496eec8b470c0
Successfully built markdown
  Attempting uninstall: markdown
    Found existing installation: Markdown 2.6.11
    Uninstalling Markdown-2.6.11:
      Successfully uninstalled Markdown-2.6.11
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 1.7.1
    Uninstalling tensorflow-1.7.1:
      Successfully uninstalled tensorflow-1.7.1


ERROR: unityagents 0.4.0 has requirement protobuf==3.5.2, but you'll have protobuf 3.6.1 which is incompatible.
ERROR: unityagents 0.4.0 has requirement tensorflow==1.7.1, but you'll have tensorflow 1.2.0 which is incompatible.
ERROR: tensorboard 1.7.0 has requirement markdown>=2.6.8, but you'll have markdown 2.2.0 which is incompatible.


In [19]:
!python {mo_tf_path}

[ ERROR ]  Path to input model or saved model dir is required: use --input_model, --saved_model_dir or --input_meta_graph
